# Создание простой RAG системы

Я реализую простую RAG систему как чат с электронным психологом. То есть за данные берем известные книги по психологии

## Шаг 1: Препроцессинг данных и создание эмбедингов
План:
1. Импорт pdf файлов
2. Препроцессинг текста для эмбедингов (разделение на чанки)
3. Эмбендинг чанков текста с помощью эмбендинговой модели
4. Сохранение эмбендингов в файл

## 1. Импорт pdf файлов

In [5]:
import os
import requests

pdf_path = 'psichology books/'
books = ['Allana-Piza-YAzyk-telodvizhenij', 'Eric-Berne-games-that-people-play', 'Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ', 'Gurina-Koshenova-Rabota-psikhologa-s_detskoi', 'Psihology-Aykido', 'Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauchitsya-ubezhdat-y-dobivatsya-uspeha']

for book in books:
    if os.path.exists(pdf_path + book + '.pdf'):
        print(f'[INFO] File exists. Skipping {book}...')
    else:
        print(f'[INFO] {book} is not found')



[INFO] File exists. Skipping Allana-Piza-YAzyk-telodvizhenij...
[INFO] File exists. Skipping Eric-Berne-games-that-people-play...
[INFO] File exists. Skipping Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ...
[INFO] File exists. Skipping Gurina-Koshenova-Rabota-psikhologa-s_detskoi...
[INFO] File exists. Skipping Psihology-Aykido...
[INFO] File exists. Skipping Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauchitsya-ubezhdat-y-dobivatsya-uspeha...


In [6]:
 #\xa0

In [7]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text : str) -> str:
    """Performs basic text formatting."""
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(file_path: str) -> list[str]:
    doc = fitz.open(file_path)
    pages_and_texts = []
    for page_num, page in tqdm(enumerate(doc)):
        text = page.get_text("text")
        text = text_formatter(text=text)
        if "Goleman" in file_path:
            text = text.replace("\xa0", " ")
        pages_and_texts.append({"book": file_path[17:-4], "page_num": page_num, "page_char_count": len(text), "page_word_count": len(text.split()), "page_token_count": len(text)/4, "text": text})
    return  pages_and_texts

texts = []
for book in books:
    texts.extend(open_and_read_pdf(pdf_path + book + '.pdf'))
len(texts)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

1748

In [8]:
# Дополнительный препроцессинг текста

import re
from nltk import sent_tokenize  # Assuming NLTK is available or can be used; if not, implement custom sentence splitter

# Additional preprocessing steps:
# 1. Hyphen removal for word breaks: Merge words split by hyphens across lines, e.g., "паци- ентам" -> "пациентам". This uses regex to find patterns like "word- " followed by another word and joins them without the hyphen.
# 2. Multiple spaces normalization: Replace multiple whitespace characters with a single space to clean up extra gaps.
# 3. Remove special annotations: Strip out common footnote markers like "— Примеч. пер." using regex patterns for known formats.
# 4. Sentence tokenization: Split the cleaned text into individual sentences using NLTK's sent_tokenize, which handles Russian punctuation reasonably well. If NLTK isn't available, a custom regex-based splitter is provided as fallback.
# 5. Lowercase normalization (optional): Convert text to lowercase for consistency, but can be skipped if case sensitivity is needed.
# 6. Trim leading/trailing punctuation: Remove any leading or trailing non-alphabetic characters from sentences if they don't belong.

def advanced_text_formatter(text: str) -> str:
    """Enhanced text formatting beyond basic."""
    # Step 1: Remove hyphens from word breaks
    text = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', text)
    
    # Step 2: Normalize multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Step 3: Remove specific annotations like footnotes
    text = re.sub(r'—\s*Примеч\.\s*пер\.\s*', '', text)  # Target "— Примеч. пер."
    text = re.sub(r'\(\s*\d+\s*\)\.', '', text)  # Remove numbered lists like "(1).", "(2)." etc.
    
    # Step 4: Optional lowercase
    # text = text.lower()  # Uncomment if needed
    
    return text.strip()

# Fallback sentence splitter if NLTK not available
def custom_sent_tokenize(text: str) -> list[str]:
    """Custom regex-based sentence tokenizer for Russian text."""
    sentence_end = re.compile(r'(?<!\w\.\w.)(?<![A-ZА-Я][a-zа-я]\.)(?<=\.|\?|\!|\…)\s')
    sentences = sentence_end.split(text)
    return [s.strip() for s in sentences if s.strip()]

# Apply advanced formatting to existing texts
for item in texts:
    item['text'] = advanced_text_formatter(item['text'])

# Now split into sentences
sentences = []
for item in texts:
    # Use NLTK if available
    try:
        page_sentences = sent_tokenize(item['text'], language='russian')
    except:
        page_sentences = custom_sent_tokenize(item['text'])
    
    for sent in page_sentences:
        # Step 6: Trim leading/trailing punctuation if unnecessary
        sent = re.sub(r'^[^\w]+', '', sent)  # Remove leading non-word chars
        sent = re.sub(r'[^\w]+$', '', sent)  # Remove trailing non-word chars
        
        sentences.append({
            "book": item['book'],
            "page_num": item['page_num'],
            "sentence": sent,
            "sent_char_count": len(sent),
            "sent_word_count": len(sent.split()),
            "sent_token_count": len(sent) / 4
        })

print(f"Total sentences extracted: {len(sentences)}")

# Optionally, save to file or further process
import json
with open('processed_sentences.json', 'w', encoding='utf-8') as f:
    json.dump(sentences, f, ensure_ascii=False, indent=4)

Total sentences extracted: 27035


In [9]:
texts[700]['text']

'Психика и медицина 281 Глава 11 Психика и медицина «Кто научил вас этому всему, доктор?» Ответ последовал незамедлительно: «Страдание». Альбер Камю . Чума Непонятная тупая боль в паху погнала меня в больницу. Осмотр не выявил ничего необычного, пока врач не увидел результаты анализа мочи: в ней были обнаружены следы крови. — Я хочу, чтобы вы легли в больницу и прошли коекакие исследования. Надо проверить работу почек, сделать цитологию… — сказал он деловым тоном. Не помню, что он говорил дальше. Мое сознание, казалось, застыло на слове «цитология». Рак. У меня осталось смутное воспоминание, что он объяснял, где и когда я должен пройти диагностику. Простейшие указания, но даже их пришлось просить несколько раз повторить. «Цитология…» — мой ум не желал расставаться с этим словом. Оно вызвало у меня такое чувство, будто меня сзади схватили за горло и грабят на пороге собственного дома.'

In [10]:
import pandas as pd

df = pd.DataFrame(texts)
df.head()


,book,page_num,page_char_count,page_word_count,page_token_count,text
0,Allana-Piza-YAzyk-telodvizhenij,0,0,0,0.00,
1,Allana-Piza-YAzyk-telodvizhenij,1,1378,182,344.50,Annotation Книга Аллана Пиза «Язык телодвижени...
2,Allana-Piza-YAzyk-telodvizhenij,2,1021,138,255.25,Различие Пространственных Зон у Горожан и Жите...
3,Allana-Piza-YAzyk-telodvizhenij,3,1052,142,263.00,"Жест Скрещивание Рук, Усиленное Сжатием Пальце..."
4,Allana-Piza-YAzyk-telodvizhenij,4,902,112,225.50,"Жесты, Используемые Мужчинами при Ухаживании Ж..."


In [11]:
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count
count,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90
std,135.44,618.48,93.74,154.62
min,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56
50%,161.00,1939.00,264.00,484.75
75%,275.00,2102.00,288.00,525.50
max,544.00,3273.00,1386.00,818.25


In [12]:
from spacy.lang.ru import Russian
nlp = Russian()

nlp.add_pipe("sentencizer")

doc = nlp('Это первое предложение. Это второе предложение! А это третье предложение?')
assert len(list(doc.sents)) == 3

list(doc.sents)

[Это первое предложение., Это второе предложение!, А это третье предложение?]

In [13]:
for item in tqdm(texts):
    item['sentences'] = list(nlp(item['text']).sents)

    item['sentences'] = [str(sent) for sent in item['sentences']]

    item['page_sentence_count_spacy'] = len(item['sentences'])

  0%|          | 0/1748 [00:00<?, ?it/s]

In [14]:
import random
random.sample(texts, k=1)

[{'book': 'Gurina-Koshenova-Rabota-psikhologa-s_detskoi',
  'page_num': 120,
  'page_char_count': 1957,
  'page_word_count': 237,
  'page_token_count': 489.25,
  'text': '121 Дети до трехлетнего возраста, как правило, будут демонстрировать следующий перечень: малоподвижность и невыраженную реакцию на внешние стимулы (применимо к младенцам), страх перед родителями и другими взрослыми, удрученный внешний вид, агрессивность, капризность, чрезмерную настороженность. Дети дошкольного возраста, как правило, будут демонстрировать следующий перечень: пассивность и излишнюю уступчивость, заискивающее поведение, агрессивность по отношению к людям и животным, воровство, вранье. Дети младшего школьного возраста, как правило, будут демонстрировать следующий перечень: желание как можно дольше находиться вне дома, низкая успеваемость в школе, трудности в концентрации внимания, воровство, агрессивность, замкнутость, желание скрыть причины имеющихся у них повреждений, трудности в общении со сверстникам

In [15]:
data = pd.DataFrame(texts)
data.describe().round(2)


,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy
count,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,14.68
std,135.44,618.48,93.74,154.62,8.53
min,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00
50%,161.00,1939.00,264.00,484.75,14.00
75%,275.00,2102.00,288.00,525.50,18.00
max,544.00,3273.00,1386.00,818.25,73.00


In [16]:
# chunking with spacy sentences (1 chunk = ?10? sentences). 

num_sentences_per_chunk = 10

def split_list(input_list: list[str], slice_size: int=num_sentences_per_chunk) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [17]:
import re


all_sentences = []
for item in tqdm(texts):
    for sent in item['sentences']:
        all_sentences.append({
            "sentence": str(sent),
            "book": item['book'],
            "page_num": item['page_num']
        })


sentence_chunks = split_list(all_sentences, slice_size=num_sentences_per_chunk)


pages_and_chunks = []
for chunk in tqdm(sentence_chunks):
    chunk_dict = {}
    chunk_dict['book'] = chunk[0]['book']
    chunk_dict['page_num'] = chunk[0]['page_num']
    
    # Join the sentences in the chunk
    joined_sentence_chunk = " ".join([item['sentence'] for item in chunk]).replace("  ", " ").strip()
    joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
    chunk_dict['sentence_text'] = joined_sentence_chunk
    
    # Recalculate stats for the new chunk
    chunk_dict['chunk_char_count'] = len(joined_sentence_chunk)
    chunk_dict['chunk_word_count'] = len(joined_sentence_chunk.split(" "))
    chunk_dict['chunk_token_count'] = len(joined_sentence_chunk) / 4
    
    pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1748 [00:00<?, ?it/s]

  0%|          | 0/2566 [00:00<?, ?it/s]

2566

In [18]:
random.sample(texts, k=1)

[{'book': 'Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ',
  'page_num': 324,
  'page_char_count': 1975,
  'page_word_count': 281,
  'page_token_count': 493.75,
  'text': 'Семейный плавильный тигель 325 Такие дети уже понимают, что значат одобрение и поддержка взрослых, и, можно надеяться, успешно справятся с испытаниями, которые встретятся им на жизненном пути. В противоположность им дети, в семьях которых царят уныние, беспорядок и невнимание, берутся за решение тех же несложных задач так, словно заранее предвидят неудачу. И хотя это не означает, что им не удастся правильно сложить кубики (они ведь понимают инструкцию, и с координацией движений у них тоже все в порядке), когда они справляются с заданием, то, по словам Бразелтона, все равно имеют «жалкий вид», словно говорят: «Я никуда не гожусь. Вы же видите, у меня так ничего и не вышло». Эти дети, скорее всего, пойдут по жизни с менталитетом пораженца, не ожидая ни поощрения, ни интереса со стороны учителе

In [19]:
df = pd.DataFrame(texts)
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy
count,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,14.68
std,135.44,618.48,93.74,154.62,8.53
min,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00
50%,161.00,1939.00,264.00,484.75,14.00
75%,275.00,2102.00,288.00,525.50,18.00
max,544.00,3273.00,1386.00,818.25,73.00


In [20]:
import re
from tqdm.auto import tqdm

num_sentences_per_chunk = 10
def split_list(input_list: list, slice_size: int = num_sentences_per_chunk) -> list[list]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(texts):
    if 'sentences' in item:
        item['sentence_chunks'] = split_list(item['sentences'])
        item['num_chunks'] = len(item['sentence_chunks'])

pages_and_chunks = []
for item in tqdm(texts):
    if 'sentence_chunks' in item:
        for sentence_chunk in item['sentence_chunks']:
            chunk_dict = {}
            chunk_dict['book'] = item['book']
            chunk_dict['page_num'] = item['page_num']
            joined_sentence_chunk = ' '.join(map(str, sentence_chunk)).replace("  ", " ").strip()
            joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
            chunk_dict['sentence_text'] = joined_sentence_chunk
            
            chunk_dict['chunk_char_count'] = len(joined_sentence_chunk)
            chunk_dict['chunk_word_count'] = len(joined_sentence_chunk.split(" "))
            chunk_dict['chunk_token_count'] = len(joined_sentence_chunk)/4

            pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1748 [00:00<?, ?it/s]

  0%|          | 0/1748 [00:00<?, ?it/s]

3427

In [44]:
random.sample(pages_and_chunks, k=1)

[{'book': 'Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauchitsya-ubezhdat-y-dobivatsya-uspeha',
  'page_num': 72,
  'sentence_text': 'При всей своей эффективности методика «отказзатемотступление» не лишена недостатков. Жертвы данной стратегии могут вознегодовать, оказавшись загнанными в угол и вынужденными подчиняться. Возмущение может проявиться двумя способами. Вопервых, жертва может решить проигнорировать словесное соглашение с тем, кто предъявляет требования. Вовторых, жертва может потерять доверие к манипулирующему требующему партнеру, решить никогда больше не иметь с ним дела. Если бы любой из этих вариантов или оба сразу начали встречаться с определенной частотой, требующему следовало бы серьезно задуматься над целесообразностью использования методики «отказзатемотступление». Однако исследования показывают, что подобные реакции при применении данной методики относительно редки и не имеют тенденции к учащению. Кроме того, поразительно, но, похоже, на самом деле они имеют место все

In [22]:
df = pd.DataFrame(texts)
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy,num_chunks
count,1748.00,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,14.68,1.96
std,135.44,618.48,93.74,154.62,8.53,0.88
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00,1.00
50%,161.00,1939.00,264.00,484.75,14.00,2.00
75%,275.00,2102.00,288.00,525.50,18.00,2.00
max,544.00,3273.00,1386.00,818.25,73.00,8.00


In [23]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_num,chunk_char_count,chunk_word_count,chunk_token_count
count,3427.00,3427.00,3427.00,3427.00
mean,191.52,862.12,118.67,215.53
std,142.89,505.03,67.04,126.26
min,0.00,2.00,1.00,0.50
25%,70.50,453.00,63.00,113.25
50%,159.00,806.00,114.00,201.50
75%,286.00,1260.00,173.00,315.00
max,543.00,2214.00,293.00,553.50


In [24]:
min_token_length = 30
for row in df[df['chunk_token_count'] <= min_token_length].sample(5).iterrows():

    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_text"]}')

Chunk token count: 29.5 | Text: Сложит два кубика, а затем поднимет на вас сияющий, полный надежды взгляд: «Ну, скажи же мне, какой я замечательный!»4
Chunk token count: 12.25 | Text: появится чувство успокоения и уверенности в себе.
Chunk token count: 5.25 | Text: И однажды, когда жена
Chunk token count: 15.5 | Text: Искусство общения 223 Часть III Эмоциональный разум в действии
Chunk token count: 21.0 | Text: Наилучшие результаты достигаются, когда школьные уроки координируются с происходящим


In [25]:
pages_and_chunks_over_min_token_len = df[df['chunk_token_count'] > min_token_length].to_dict(orient='records')
pages_and_chunks_over_min_token_len[20:25]

[{'book': 'Allana-Piza-YAzyk-telodvizhenij',
  'page_num': 17,
  'sentence_text': 'Конгруэнтность — Совпадение Слов и Жестов Если бы вы были собеседником человека, показанного на рис. 4, и попросили его выразить свое мнение относительно того, что вы только что сказали, на что он бы ответил, что с вами не согласен, то его невербальные сигналы были бы конгруэнтными, т.е. соответ-ствовали бы его словесным высказываниям. Если же он скажет, что ему очень нравится все, что вы говорите, он будет лгать, потому что его слова и жесты будут не конгруэнтными. Исследования доказывают, что невербальные сигналы несут в 5 раз больше информации, чем вербальные, и в случае, если сигналы не — конгруэнтны, люди полагаются на невербальную информацию, предпочитая ее словесной. Часто можно наблюдать, как какой-нибудь политик стоит на трибуне, крепко скрестив руки на груди ( защитная поза ) с опущенным подбородком ( критическая или враждебная поза), и говорит аудитории о том, как восприимчиво и дружелюбно он 

In [26]:
random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'book': 'Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ',
  'page_num': 58,
  'sentence_text': 'Анатомия «захвата эмоций» 59 ситуации, требующей мгновенной реакции. Однако по цепочке от таламуса к миндалевидному телу передается только небольшая часть сенсорной информации, а бóльшая проходит по главному пути — к неокортексу. Так что в миндалевидное тело по экспрессмаршруту в лучшем случае поступает простой сигнал в качестве предостережения. Как отмечал Джозеф Леду: «Не нужно точно знать, что случилось, достаточно понимать, что это может быть опасно»10. Прямой проводящий путь имеет огромное преимущество, потому что мозг в данном случае реагирует за тысячные доли секунды. К примеру, миндалевидное тело мозга крысы запускает ответную реакцию на восприятие менее чем через двенадцать миллисекунд, то есть через двенадцать тысячных секунды. Путь от таламуса к неокортексу, а от него к миндалевидному телу занимает примерно в два раза больше времени. Аналогичные измерени

## Эмбендинг

In [83]:
import requests

model_id = "sentence-transformers/all-MiniLM-L6-v2" #hugging face model
hf_token = "get your token in http://hf.co/settings/tokens"

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path=model_id, device='cpu')

sentences = ["Своевременное управление своими эмоциями помогает человеку лучше адаптироваться в обществе и достигать успехов в различных сферах жизни.", 
             "Эмоциональный интеллект включает в себя способность распознавать, понимать и управлять своими эмоциями, а также эмоциями других людей.",
             "Мне нравятся лошади"]




embeddings = embedding_model.encode(sentences)
embeddings_dictionary = dict(zip(sentences, embeddings))

for sentence, embedding in embeddings_dictionary.items():
    print(f'Sentence: {sentence}\nEmbedding: {embedding[:5]}\n')

c:\Users\Delta-Game\first_rag\venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Sentence: Своевременное управление своими эмоциями помогает человеку лучше адаптироваться в обществе и достигать успехов в различных сферах жизни.
Embedding: [ 0.03206125  0.00712634 -0.03242809 -0.06177649 -0.08451619]

Sentence: Эмоциональный интеллект включает в себя способность распознавать, понимать и управлять своими эмоциями, а также эмоциями других людей.
Embedding: [-0.0020538   0.0575527   0.00068112 -0.03272427 -0.06679673]

Sentence: Мне нравятся лошади
Embedding: [ 0.01878012  0.05660878  0.0309769   0.05738923 -0.05720469]



In [84]:
embeddings[0].shape

(384,)

%%time

embedding_model.to('cpu')

for item in tqdm(pages_and_chunks_over_min_token_len):
    item['embedding'] = embedding_model.encode(item['sentence_text'])

In [85]:
%%time

embedding_model.to('cuda')

for item in tqdm(pages_and_chunks_over_min_token_len):
    item['embedding'] = embedding_model.encode(item['sentence_text'])

  0%|          | 0/3245 [00:00<?, ?it/s]

CPU times: total: 29.2 s
Wall time: 27.5 s


In [86]:
%%time
embedding_model.to('cuda')

text_chunks = [item["sentence_text"] for item in pages_and_chunks_over_min_token_len]
text_chunks[419]

CPU times: total: 0 ns
Wall time: 2 ms


'Глава 5. Игры 55 B. Наконец, игра третьей степени — это игра безудержная; доведённая до конца, она завершается в больнице, в суде или морге. Игры можно классифицировать также и по другим специфическим признакам, указанным при анализе ВИТ: по целям, ролям, наиболее очевидным преимуществам. Для систематической научной классификации наиболее подходящей может оказаться, вероятно, экзистенциальная точка зрения; но поскольку этот фактор ещё недостаточно изучен, такую классификацию придётся отложить на будущее. За неимением её, в настоящее время самой удобной классификацией представляется социологическая, которой мы и будем придерживаться в следующей части. Примечания Необходимо отметить заслуги Стивена Поттера с его чутким, пронизанным юмором анализом манёвров или “проделок” в повседневных жизненных ситуациях [2], и Дж. Г. Мида, пионера в исследовании общественной роли игр [3]. Игры, ведущие к психиатрическим расстройствам, систематически изучались на Сан-Францисских семинарах по социальной

In [87]:
len(text_chunks)

3245

In [88]:
%%time

text_chunk_embeddings = embedding_model.encode(text_chunks, batch_size=32, convert_to_tensor=True)

text_chunk_embeddings

CPU times: total: 9.73 s
Wall time: 4.11 s


tensor([[-0.0225, -0.0305,  0.0061,  ...,  0.0219, -0.0526, -0.0749],
        [-0.0054, -0.0366, -0.0129,  ...,  0.0367, -0.0350, -0.0389],
        [-0.0090, -0.0283, -0.0434,  ..., -0.0301, -0.0613, -0.0600],
        ...,
        [-0.0087, -0.0332, -0.0073,  ...,  0.0140,  0.0027, -0.0322],
        [-0.0221, -0.0040, -0.0072,  ..., -0.0022,  0.0087, -0.0638],
        [-0.0185, -0.0285, -0.0438,  ..., -0.0423,  0.0174,  0.0105]],
       device='cuda:0')

## Сохранение эмбеддингов в файл

In [89]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = 'text_chunks_and_embeddings_df.csv'
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [90]:
embeddings_df_save_path = 'text_chunks_and_embeddings_df.csv'
text_chunks_and_embeddings_df = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df.head()

,book,page_num,sentence_text,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,Allana-Piza-YAzyk-telodvizhenij,1,Annotation Книга Аллана Пиза «Язык телодвижени...,1377,183,344.25,[-2.24940646e-02 -3.05289645e-02 6.08610874e-...
1,Allana-Piza-YAzyk-telodvizhenij,2,Различие Пространственных Зон у Горожан и Жите...,1021,138,255.25,[-5.37875900e-03 -3.65891531e-02 -1.28565012e-...
2,Allana-Piza-YAzyk-telodvizhenij,3,"Жест Скрещивание Рук, Усиленное Сжатием Пальце...",1052,142,263.00,[-9.00302548e-03 -2.82603092e-02 -4.34474126e-...
3,Allana-Piza-YAzyk-telodvizhenij,4,"Жесты, Используемые Мужчинами при Ухаживании Ж...",893,112,223.25,[ 1.56302750e-02 -4.22257856e-02 2.18075067e-...
4,Allana-Piza-YAzyk-telodvizhenij,5,Как Ступени Ног Выражают Заинтересованность Ра...,764,103,191.00,[-4.12002280e-02 -4.65158224e-02 3.52464616e-...


Если бы у меня была действительно большая база данных эмбэндингов (100K - 1M) то тогда бы использовала векторную базу данных qdrant

# Поиск и ответы

In [91]:
import random
import torch
import numpy as np
import pandas as pd
device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embeddings_df = pd.read_csv("text_chunks_and_embeddings_df.csv")


text_chunks_and_embeddings_df['embedding'] = text_chunks_and_embeddings_df['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=' '))

embeddings = torch.tensor(np.stack(text_chunks_and_embeddings_df['embedding'].tolist(), axis=0), dtype=torch.float32).to(device)

pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient='records')

text_chunks_and_embeddings_df

,book,page_num,sentence_text,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,Allana-Piza-YAzyk-telodvizhenij,1,Annotation Книга Аллана Пиза «Язык телодвижени...,1377,183,344.25,"[-0.0224940646, -0.0305289645, 0.00608610874, ..."
1,Allana-Piza-YAzyk-telodvizhenij,2,Различие Пространственных Зон у Горожан и Жите...,1021,138,255.25,"[-0.005378759, -0.0365891531, -0.0128565012, -..."
2,Allana-Piza-YAzyk-telodvizhenij,3,"Жест Скрещивание Рук, Усиленное Сжатием Пальце...",1052,142,263.00,"[-0.00900302548, -0.0282603092, -0.0434474126,..."
3,Allana-Piza-YAzyk-telodvizhenij,4,"Жесты, Используемые Мужчинами при Ухаживании Ж...",893,112,223.25,"[0.015630275, -0.0422257856, 0.0218075067, -0...."
4,Allana-Piza-YAzyk-telodvizhenij,5,Как Ступени Ног Выражают Заинтересованность Ра...,764,103,191.00,"[-0.041200228, -0.0465158224, 0.00352464616, -..."
...,...,...,...,...,...,...,...
3240,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,403,"Journal of Personality and Social Psychology, ...",742,108,185.50,"[0.0247699656, 0.0792248473, -0.0176644269, 0...."
3241,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,404,"University, Institute for Research in the Beha...",789,111,197.25,"[0.00855329819, 0.0199707057, -0.0688934103, 0..."
3242,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,405,убийства в Вирджинии и Северном Иллинойсе). 2....,955,124,238.75,"[-0.00873123482, -0.0332376324, -0.00725876074..."
3243,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,405,"Удвоившееся количество отчетов читателей, их р...",631,82,157.75,"[-0.0221029315, -0.00398574863, -0.00724793691..."


In [92]:
random.sample(list(text_chunks_and_embeddings_df['sentence_text']), k=1)

['В то же время эти родители подавали яркий (и вопиющий) пример агрессивности — образец, которому их дети следовали всю жизнь начиная со школы или игровой площадки. При этом родители не обязательно были злодеями или не желали детям самого лучшего — видимо, они просто воспроизводили стиль воспитания, смоделированный для них собственными родителями. При такой модели обращения с детьми наказания возникают сообразно прихоти родителей. Если они пребывают в дурном расположении духа, то дети получают суровую трепку, если же у них хорошее настроение, то детям удается избежать очередной жестокой расправы. Получается, наказание подчас зависит не от того, какой именно проступок совершил ребенок, а от настроения родителей. И вот вам']

In [93]:
embeddings.shape

torch.Size([3245, 384])

In [98]:
from sentence_transformers import SentenceTransformer, util
embedding_model = SentenceTransformer(model_name_or_path=model_id, device='cpu')

c:\Users\Delta-Game\first_rag\venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [99]:
from time import perf_counter as timer

query = 'Здоровые отношения'
print(f'Query: {query}')

query_embedding = embedding_model.encode(query, convert_to_tensor=True).to(device)

print(query_embedding.shape)


cosine_scores = util.cos_sim(query_embedding, embeddings)[0]

start_time = timer()
dot_scores = util.dot_score(query_embedding, embeddings)[0]
end_time = timer()

print(f'[INFO] Time taken to compute dot product scores: {end_time - start_time:.4f} seconds \n len = {len(embeddings)}')

top_results_of_the_dot_product = torch.topk(dot_scores, k=5)
top_results_of_the_dot_product

Query: Здоровые отношения
torch.Size([384])
[INFO] Time taken to compute dot product scores: 0.0001 seconds 
 len = 3245


torch.return_types.topk(
values=tensor([0.7320, 0.7042, 0.7038, 0.6992, 0.6732], device='cuda:0'),
indices=tensor([1917, 1444,  649,   49, 2020], device='cuda:0'))

In [100]:
for i in top_results_of_the_dot_product.indices.tolist():
    print(text_chunks_and_embeddings_df['sentence_text'][i])

D. Изменения в возбуждении и реактивности, ассоциированные с травматическим событием (событиями), начинаются после действия
Таким образом проводится своего рода коррекция ограниченного мышления, при котором конфликт рассматривается как единственный путь к урегулированию разногласий.
1“Общественные отношения” — система организации контактов между людьми в торговле и промышленности, распространённая в США и имеющая целью создание благоприятных психологических условий для бизнеса.
Датчане же думали, что американцы были холодны и недружелюбны, потому что они устранялись от удобной для их общения зоны.
196 Практическое задание Соотнесите фотографии песочных картин детей со стадиями работы: 1. A. Стадия хаоса 2. B. Стадия борьбы 3. C. Стадия исхода D. Стадия контроля


In [101]:
larger_embeddings = torch.randn(1000*embeddings.shape[0], 768).to(device)
print(f"Larger embeddings shape: {larger_embeddings.shape}")

start_time = timer()
dot_scores = util.dot_score(query_embedding, larger_embeddings)[0]
end_time = timer()

print(f'[INFO] Time taken to compute dot product scores with larger embeddings: {end_time - start_time:.4f} seconds \n len = {len(larger_embeddings)}')

Larger embeddings shape: torch.Size([3245000, 768])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x384 and 768x3245000)

In [ ]:
import textwrap

def print_wrapped(text: str, width: int=80):
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

In [ ]:
print(f"Query: {query}\n")
print("Results:")

for score, idx in zip(top_results_of_the_dot_product[0], top_results_of_the_dot_product[1]):
    print_wrapped(f"Score: {score:.4f} | Text: {pages_and_chunks[idx]['sentence_text']}\n")


Query: Здоровые отношения между людьми способствуют их эмоциональному благополучию и личностному росту.

Results:
Score: 0.8361 | Text: Возможно, лучшей иллюстрацией опыта, который способен
изменить темперамент в лучшую сторону, служат результаты проведенных Кейганом
экспериментов с участием застенчивых детей.
Score: 0.8087 | Text: Теперь при обучении летчиков наряду с техническим
мастерством особое внимание уделяется взаимодействию, открытой коммуникации,
Score: 0.8010 | Text: И такая бесстрастная манера рассуждать логически, по
мнению Дамасио, составляла суть проблемы Эллиота: неспособность понять
собственные чувства, возникающие по поводу разных обстоятельств, вносила ошибку
в его рассуждения.
Score: 0.7972 | Text: М.Л.) Комментировать этот диалог достаточно просто. Здесь
легко просматриваются приемы амортизации непосредственной и профилактической.
Заслуживает разбора лишь
Score: 0.7829 | Text: Но они же способны легко сбить нас с пути истинного, что
часто и происходит. Как представ